## Cryptography 1 - Week 4 Programming Assignment 
###Brandon Kinman, 5/29/2015###

###Problem Description

Suppose an attacker wishes to steal secret information from our target web site crypto-class.appspot.com. The attacker suspects that the web site embeds encrypted customer data in URL parameters such as this:

http://crypto-class.appspot.com/po?er=f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4

That is, when customer Alice interacts with the site, the site embeds a URL like this in web pages it sends to Alice. The attacker intercepts the URL listed above and guesses that the ciphertext following the "po?er=" is a hex encoded AES CBC encryption with a random IV of some secret data about Alice's session.

After some experimentation the attacker discovers that the web site is vulnerable to a CBC padding oracle attack. In particular, when a decrypted CBC ciphertext ends in an invalid pad the web server returns a 403 error code (forbidden request). When the CBC padding is valid, but the message is malformed, the web server returns a 404 error code (URL not found).

Armed with this information your goal is to decrypt the ciphertext listed above. To do so you can send arbitrary HTTP requests to the web site of the form

http://crypto-class.appspot.com/po?er="your ciphertext here"

and observe the resulting error code. The padding oracle will let you decrypt the given ciphertext one byte at a time. To decrypt a single byte you will need to send up to 256 HTTP requests to the site. Keep in mind that the first ciphertext block is the random IV. The decrypted message is ASCII encoded.

###Solution
This solution is based uopn an example script given to us by profressor Boneh.

In [10]:
import urllib2
import sys
import array

TARGET = 'http://crypto-class.appspot.com/po?er='
BLOCK_SIZE = 16
ctString = 'f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4'
ct = ctString.decode('hex')

class PaddingOracle(object):
    def query(self, q):
        target = TARGET + urllib2.quote(q)    # Create query URL
        req = urllib2.Request(target)         # Send HTTP request to server
        try:
            f = urllib2.urlopen(req)          # Wait for response
        except urllib2.HTTPError, e:          
            #print "We got: %d" % e.code       # Print response code
            print('.'),
            if e.code == 404:
                return True # good padding
            return False # bad padding
        
def strxor(a, b):     # xor two strings of different lengths
    if len(a) > len(b):
        return "".join([chr(ord(x) ^ ord(y)) for (x, y) in zip(a[:len(b)], b)])
    else:
        return "".join([chr(ord(x) ^ ord(y)) for (x, y) in zip(a, b[:len(a)])])
        
def padForLen(len):
    return array.array('B', [x^x^len for x in range(0,len)]).tostring()

po = PaddingOracle()

plainText = [];

for j in range(len(ct)-1,BLOCK_SIZE-1,-1):
    chainBytes = ct[j-BLOCK_SIZE:((j-BLOCK_SIZE)/BLOCK_SIZE)*BLOCK_SIZE+BLOCK_SIZE]
    pad = padForLen(BLOCK_SIZE-j%BLOCK_SIZE)
    for guessByte in range(0,256):
        guess = chr(guessByte)+''.join(plainText)
        modifiedBytes = strxor(strxor(chainBytes[::-1],pad[::-1]),guess[::-1])[::-1]
        query = ct[0:len(ct)-len(modifiedBytes)]+modifiedBytes
        
        if guessByte == 0:
            print('chainbytes:\t'+chainBytes.encode('hex'))
            print('pad:\t'+pad.encode('hex'))
            print('guess:\t'+guess.encode('hex'))
            print('XORED:\t'+modifiedBytes.encode('hex'))
            print('query:\t'+query[len(query)-16:len(query)].encode('hex'))

        if j >=61:
            if po.query(query.encode('hex')):
                print('\n Plaintext recovered:'+chr(guessByte).encode('hex'))
                plainText.insert(0,chr(guessByte))
                break
        #print('query:'+query.encode('hex'))
        #padForLen(numPadBytes)  
        
#print(po.query(ciphertext))

chainbytes:	c0
pad:	01
guess:	00
XORED:	c1
query:	bdf302936266926ff37dbf7035d5eec1
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
 Plaintext recovered:3e
chainbytes:	77c0
pad:	0202
guess:	003e
XORED:	75fc
query:	bdf302936266926ff37dbf7035d575fc
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . chainbytes:	a577c0
pad:	030303
guess:	003e
XORED:	a64a
query:	bdf302936266926ff37dbf7035d5a64a
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [115]:
'\x70'

'p'

In [110]:
'\x3e'

'>'

In [27]:
a = []

In [33]:
a.insert(0,3)
a

[3, 2, 2]

In [11]:
chain = 'c0'.decode('hex')
pad = '01'.decode('hex')
guess = '09'.decode('hex')
strxor(strxor(chain,pad),guess).encode('hex')

'c8'

In [12]:
result = po.query('f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c8bdf302936266926ff37dbf7035d5eeb4')
print(result)

. True


In [ ]:
po.query(query.encode('hex')